In [4]:
from model import ParticleClassifier
import numpy as np

In [ ]:
classifier = ParticleClassifier()\
                .load_data('data/toy_data.csv')\
                .train_test_split()\
                .pre_proc_images(train=True, test=True)\
                .one_hot_encode_labels(train=True, test=True)\
                .train_model()\
                .evaluate_model()\

In [ ]:
# Hot pixel attack
classifier.hot_pixel_attack()\
          .pre_proc_images(attacked=True)\
          .one_hot_encode_labels(attacked=True)\
          .evaluate_attack()

In [6]:
# Background Neutrons (2x2) square of relatively high value
classifier = ParticleClassifier()\
                .load_data('data/toy_data.csv')\
                .train_test_split()\
                .pre_proc_images(train=True, test=True, filters=False)\
                .one_hot_encode_labels(train=True, test=True)\
                .train_model()\
                .evaluate_model()
value =  np.mean([classifier.images.max(), classifier.images.mean()])
classifier = classifier.apply_attack(classifier.add_hot_area, size=(2,2), value=value)\
                       .pre_proc_images(attacked=True, filters=False)\
                       .one_hot_encode_labels(attacked=True)\
                       .evaluate_attack()

Instructions for updating:
Colocations handled automatically by placer.
Data loaded into self.images, self.labels
Split into self.images_train, self.images_test, self.labels_train, self.labels_test
Train and test labels encoded into self.y_train and self.y_test
Train on 1920 samples, validate on 480 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1920/1920 [==============================] - 7s 3ms/sample - loss: 0.5708 - acc: 0.7219 - val_loss: 0.3238 - val_acc: 0.8771
Epoch 2/10
1920/1920 [==============================] - 5s 3ms/sample - loss: 0.2977 - acc: 0.8771 - val_loss: 0.3238 - val_acc: 0.8604
Epoch 3/10
1920/1920 [==============================] - 5s 3ms/sample - loss: 0.2541 - acc: 0.9005 - val_loss: 0.2266 - val_acc: 0.9042
Epoch 4/10
1920/1920 [==============================] - 5s 3ms/sample - loss: 0.2358 - acc: 0.9109 - val_loss: 0.1859 - val_acc: 0.9333
Epoch 5/10
1920/1920 [==============================] - 5s 3ms/sample - loss: 0.1601 - acc: 0.9453 

/Users/shawn/miniconda3/envs/test/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
images = classfier.images
value =  np.mean([classifier.images.max(), classifier.images.mean()])
classifier.apply_attack(classifier.add_hot_area, size=(2,2), value=value)

In [ ]:
np.mean([classifier.images.max(), classifier.images.mean()])